In [ ]:
import pandas as pd
import numpy as np
import mpxj
import jpype
import datetime

import plotly.express as px

%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 2000

PATH = '/Users/antonmorozov/'

import os
import sys

print('Path of working directory  %s' % os.getcwd())
print('Path of the executable binary %s' % sys.executable)
sys.version

In [ ]:
df = pd.read_csv(os.path.join(PATH, 'downloads', 'gannt.csv'))

In [ ]:
df = pd.read_csv(os.path.join(PATH, 'downloads', 'S03_CGM|D15_time_exp.csv'))
df = df[['Тип данных', 'Идентификатор', 'Сложность задачи (1-10)']]
df['Сложность задачи (1-10)'].fillna(0, inplace=True)
df['Start'] = df['Сложность задачи (1-10)'].cumsum().shift(1).fillna(0)
df['Finish'] = df['Сложность задачи (1-10)'].cumsum()
df['Start-Finish'] = df['Finish'] - df['Start']

def color(row):
    "Функция для раскраски графа"
#     c_dict = {'Промежуточные':'#E64646', 'Действие':'#3475D0'}
    c_dict = {'Промежуточные':'Red', 'Действие':'Blue'}
    return c_dict[row['Тип данных']]

df['Color'] = df.apply(color, axis=1)


def convert(number,
            year=datetime.datetime.today().strftime("%m/%d/%Y").split('/')[2],
            month=datetime.datetime.today().strftime("%m/%d/%Y").split('/')[0],
            day=datetime.datetime.today().strftime("%m/%d/%Y").split('/')[1],
            hour=datetime.datetime.today().strftime("%m/%d/%Y %H:%M:%S").split(' ')[1].split(':')[0],
            minute=datetime.datetime.today().strftime("%m/%d/%Y %H:%M:%S").split(' ')[1].split(':')[1]
            ):
    
    '''
    Конвертирует number (float) – ЧИСЛО ЧАСОВ,  в полную дату с точностью до минут
    Пользователь может указать год, месяц, день, иначе назначается сегодняшняя дата
    
    TODO: добавить учет конца месяца
    '''
    day_ini = day
    to_day_hour = number/24  #  выводит float, где день - число перед запятой
    if len(str(to_day_hour).split('.')[0] + day_ini) > 1:  #  учет если кол-во дней больше 10
        day = str(int(to_day_hour)+int(day_ini))
    else:
        day = '0'+str(int(to_day_hour)+int(day_ini))
     
    hour_ini = hour
    hour = str(int(round(float('.'+str(number/24).split('.')[1])*24, 2)) + int(hour_ini))
    
    if int(hour) >= 24:
        day = str(int(day) + int(str(float(hour)/24).split('.')[0]))
        hour = str(int(round(float('.'+str(float(hour)/24).split('.')[1])*24,0)))
        if int(day) > 31:
            print('more than 31 days!')

    if len(hour) > 2:
        hour = '0'+ hour
        
    minute_ini = minute
    hour_to_min = str(round(float('.'+str(number/24).split('.')[1])*24, 5))
    minute = str(int(round(float('.'+hour_to_min.split('.')[1])*60, 2))+int(minute_ini))
    
    return pd.to_datetime(month+'/'+day+'/'+year+' '+hour+':'+minute, format='%m/%d/%Y %H:%M')

df['Start Date'] = df['Start'].to_frame().applymap(convert)
df['Finish Date'] = df['Finish'].to_frame().applymap(convert)

fig = px.timeline(df, x_start="Start Date", x_end="Finish Date", y="Идентификатор",
#                   color="Тип данных",  # не работает
#                   category_orders=dict(group=df['Идентификатор'].tolist()),
                 )
# fig.update_layout(yaxis_categoryorder = 'total ascending')
fig.update_yaxes(autorange="reversed")

In [12]:
may2017 = clnd('May 2017', period='M')
may2017.count()

20